In [1]:
!unzip /content/drive/MyDrive/yolov52.zip -d /content/output_folder

Archive:  /content/drive/MyDrive/yolov52.zip
   creating: /content/output_folder/content/yolov5/
  inflating: /content/output_folder/content/yolov5/README.zh-CN.md  
   creating: /content/output_folder/content/yolov5/.git/
   creating: /content/output_folder/content/yolov5/.git/hooks/
  inflating: /content/output_folder/content/yolov5/.git/hooks/update.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/pre-rebase.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/prepare-commit-msg.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/applypatch-msg.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/post-update.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/pre-commit.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/pre-merge-commit.sample  
  inflating: /content/output_folder/content/yolov5/.git/hooks/commit-msg.sample  
  inflating: /content/output_folder/content

In [2]:
!pip install onnx onnxruntime-gpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00


In [4]:

!pip install ultralytics -q
# Export the distilled Nano model to ONNX INT8
# This uses Post-Training Quantization (PTQ) logic which is very stable for YOLOv5n
!python /content/output_folder/content/yolov5/export.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.pt \
                  --img 1024 \
                  --batch 1 \
                  --device 0 \
                  --include onnx \
                  --int8 \
                  --data /content/output_folder/content/yolov5/data.yaml

export: data=/content/output_folder/content/yolov5/data.yaml, weights=['/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.pt'], imgsz=[1024], batch_size=1, device=0, half=False, inplace=False, keras=False, optimize=False, int8=True, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs

PyTorch: starting from /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.pt with output shape (1, 64512, 13) (4.1 MB)
requirements: Ultralytics requirement ['onnxscript'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 8 packages in 236ms
Prepared 2

In [5]:
import onnxruntime as ort
import numpy as np
import cv2

# 1. Load the model
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
session = ort.InferenceSession('/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx', providers=providers)

def preprocess(img_path, input_size=1024):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (input_size, input_size))
    img = img.transpose((2, 0, 1)) # HWC to CHW
    img = np.expand_dims(img, axis=0).astype(np.float32)
    img /= 255.0  # Normalize
    return img

# 2. Run Inference
input_name = session.get_inputs()[0].name
test_image = preprocess('/content/output_folder/content/yolov5/test/images/1700059259412_jpg.rf.86e160c4ccbaac8e6d4093ffb4cfecab.jpg')
outputs = session.run(None, {input_name: test_image})

print("Inference successful. Detected objects shape:", outputs[0].shape)

Inference successful. Detected objects shape: (1, 64512, 13)


In [ ]:
# FP32 ONNX
!python /content/output_folder/content/yolov5/val.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx \
               --data /content/output_folder/content/yolov5/data.yaml \
               --img 1024 \
               --task val \
               --batch-size 1

val: data=/content/output_folder/content/yolov5/data.yaml, weights=['/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx'], batch_size=1, imgsz=1024, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=output_folder/content/yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Loading /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx for ONNX Runtime inference...
Forcing --batch-size 1 square inference (1,3,1024,1024) for non-PyTorch models
val: Scanning /content/output_folder/content/yolov5/valid/labels... 14 images, 0 backgrounds, 0 corrupt: 100% 14/14 [00:00<00:00, 150.17it/s]
val: New cache created: /content/output_folder/content/yolov5/valid/labels.cache


In [14]:
!pip install onnxruntime-gpu onnxruntime-quantization

ERROR: Could not find a version that satisfies the requirement onnxruntime-quantization (from versions: none)
ERROR: No matching distribution found for onnxruntime-quantization


In [ ]:
# CONVERT TO INT8 ONNX
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# Paths
model_fp32 = '/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx'
model_int8 = '/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/rice_final_int8.onnx'

# Quantize weights from Float32 to Int8
quantize_dynamic(model_fp32, model_int8, weight_type=QuantType.QUInt8)

import os
size = os.path.getsize(model_int8) / (1024 * 1024)
print(f"Final Quantized Model Size: {size:.2f} MB")

Final Quantized Model Size: 2.99 MB


In [ ]:
# INT8 ONNX
!python /content/output_folder/content/yolov5/val.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/rice_final_int8.onnx \
               --data /content/output_folder/content/yolov5/data.yaml \
               --img 1024 \
               --task val \
               --batch-size 1

val: data=/content/output_folder/content/yolov5/data.yaml, weights=['/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/rice_final_int8.onnx'], batch_size=1, imgsz=1024, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=output_folder/content/yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Loading /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/rice_final_int8.onnx for ONNX Runtime inference...
2025-12-24 06:40:24.938957012 [W:onnxruntime:, transformer_memcpy.cc:111 ApplyImpl] 159 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_le

In [ ]:
# 1. Evaluate FP32 Model
!python /content/output_folder/content/yolov5/val.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx \
               --data /content/output_folder/content/yolov5/data.yaml --img 1024 --task val --batch-size 1

# 2. Evaluate INT8 Model
!python /content/output_folder/content/yolov5/val.py --weights /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/rice_final_int8.onnx \
               --data /content/output_folder/content/yolov5/data.yaml --img 1024 --task val --batch-size 1

val: data=/content/output_folder/content/yolov5/data.yaml, weights=['/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx'], batch_size=1, imgsz=1024, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=output_folder/content/yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-453-geed9bc19 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Loading /content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx for ONNX Runtime inference...
Forcing --batch-size 1 square inference (1,3,1024,1024) for non-PyTorch models
val: Scanning /content/output_folder/content/yolov5/valid/labels.cache... 14 images, 0 backgrounds, 0 corrupt: 100% 14/14 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP5

In [23]:
import time
import os
import numpy as np
import onnxruntime as ort

def benchmark_onnx(model_path):
    session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    input_name = session.get_inputs()[0].name
    dummy_input = np.random.randn(1, 3, 1024, 1024).astype(np.float32)

    # Warmup
    for _ in range(10): session.run(None, {input_name: dummy_input})

    # Speed Test
    start_time = time.time()
    iterations = 100
    for _ in range(iterations):
        session.run(None, {input_name: dummy_input})
    avg_speed = ((time.time() - start_time) / iterations) * 1000 # ms

    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    return size_mb, avg_speed

# Comparison
fp32_size, fp32_speed = benchmark_onnx('/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/best.onnx')
int8_size, int8_speed = benchmark_onnx('/content/output_folder/content/yolov5/runs/train/rice_nano_distilled_SOTA/weights/rice_final_int8.onnx')

print(f"{'Metric':<20} | {'FP32 ONNX':<15} | {'INT8 ONNX':<15}")
print("-" * 55)
print(f"{'Model Size (MB)':<20} | {fp32_size:<15.2f} | {int8_size:<15.2f}")
print(f"{'Inference (ms)':<20} | {fp32_speed:<15.2f} | {int8_speed:<15.2f}")

Metric               | FP32 ONNX       | INT8 ONNX      
-------------------------------------------------------
Model Size (MB)      | 8.06            | 2.99           
Inference (ms)       | 255.32          | 298.28         
